In [1]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, max_len=140, title="Retriever Results"):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        table.add_row(str(i), src, page, (text[:max_len] + ("…" if len(text) > max_len else "")))

    console.print(table)

## 1. 문서 로드

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf"

print("문서로드 중")

loader = PyPDFLoader(file_path)
docs = loader.load()
docs[:5]

문서로드 중


[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 0, 'page_label': '1'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 1, 'page_label': '2'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\

## 2. splitter

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunk = splitter.split_documents(docs)
len(chunk)

237

In [6]:
chunk[5:7]

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 3, 'page_label': '4'}, page_content="사회공헌 분야에서는 2024년 '삼성 청년SW·AI아카데미'에 375억 원을 \n지원해 2,200 명의 청년 소 프트웨어 인재를 양성했으며, 2025년에는 \n마이스터고 졸업생에게까지 교육기회를 확대할 예정입니다. \n또한 자립 준비 청년들의 주 거안정과 미래준비를 지원하는 '삼성 \n희망디딤돌' 활동은 2024년에 충북과 대전센터를 신규로 개소하여 총 \n14,362명을 지원하였으며, 2025년에는 인천센터를 추가로 설립하여 더 \n많은 청년들을 지원할 예정입니다.\n지난해에 이 어 올해 지속가능경영보고서도 글로벌 공시 규 제 \n프레임워크에 맞춰 발 행됐 습니다. 2024년 한 해 동안 회사의  \n지속가능경영 관리체계, 전략, 이 행 활동, 성과 등을 충실 히 담았으며, \n지속가능경영 웹사이트와도 더욱 긴밀히 연계되도록 구성했습니다.\n삼성전자는 불확실성의 시대에 재도 약의 기반을 다지기 위해 지속 \n노력하고 있습니다. '인재와 기술을 바탕으로 최고의 제품과 서비스를 \n창출하여 인류사회에 공헌한다'는 경영철학에 입각해 기술 리더십으로 \n재도약의 기반을 다지고 새로운 영 역에서 미래 성장동력을 확보해 \n나가겠습니다.\n삼성전자는 이해관계자 여러분의 소중한 의견에 늘 귀를 기울이고 있으며, \n앞으로도 지속가능한 성장 기반을 마련하는 

## 3. vectorDB 저장 및 불러오기

In [2]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [ ]:
# 벡터 저장소 만들기 위한 변수 설정
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")
persist_directory = "../vectorStore/samsung_2025_db"
collection_name = "samsung2025"

In [ ]:
# 처음 실행시 벡터 저장소 만들기
# vectorStore = Chroma.from_documents(
#     documents = chunk,
#     collection_name = collection_name,
#     persist_directory = persist_directory,
#     embedding = embedding
# )

In [10]:
# 벡터 저장소 불러오기
load_vectorStore = Chroma(
    persist_directory = persist_directory,
    collection_name = collection_name,
    embedding_function = embedding
)

## 4. Retriever

### 1) 키워드기반 + 기본검색기 = 하이브리드(Ensemble)

- 일반 retriever : 벡터 기반
- bm25 : 키워드 기반

```
uv add rank-bm25
```

In [22]:
# 유사도 기반
ret_similarity = load_vectorStore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)

In [11]:
# 번외 : 메타데이터가 궁금
db_docs = load_vectorStore._collection.get(include=["documents", "metadatas"])

In [16]:
db_docs["documents"][:3]

['삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future',
 '삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\n삼성전자 지속가능경영보고서 2025\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n준법과 윤리경영\n[DX부문] \n추진체계와 주요성과 \n기후변화 \n자원순환\n수자원\n오염물질\n경제성과\n사회성과\n환경성과 \n사업부문별 환경성과 \n[DS부문] \n추진체계와 주요성과 \n기후변화 \n자원순환\n수자원\n오염물질\n독립된 인증인의 인증보고서\nScope 1, 2 온실가스 배출량 검증 의견서 \nScope 3 온실가스 배출량 검증 의견서 \nGRI Index\nTCFD 대조표\nSASB 대조표\nAbout This Report \n임직원\n공급망\n사회공헌\n개인정보보호와 보안\n제품 품질과 안전\nOur Company\nPrinciple\nPlanet\nFacts & Figures Appendix \nPeople\n04\n05\n06\n07\n09\n59\n11\n12\n16\n18\n20\n62\n63\n68\n72\n21\n22\n27\n29\n32\n76\n77\n78\n80\n82\n84\n86\n35\n45\n51\n53\n55',
 '삼성전자 지속가능경영보고서 2025\n03\nOur Company AppendixFacts & Figures PrinciplePlanet People\nOur Company\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n

In [15]:
db_docs["metadatas"][:1]

[{'creationdate': '2025-07-10T16:11:16+09:00',
  'trapped': '/False',
  'total_pages': 87,
  'page': 0,
  'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'moddate': '2025-09-04T16:51:11+09:00',
  'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'page_label': '1',
  'producer': 'Adobe PDF Library 15.0'}]

In [17]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [18]:
# 사실은 chunk 를 쓰면 되지만
# 우리는 vectorStore를 처음에 만들고, 더이상 똑같은걸 다시 만들지 않을 것이므로
# load_vectorStore._collection 으로 가져와서 chunk 같은 Document 리스트를 만들어야 함
from langchain_core.documents import Document
bm_doc = []
for content, meta in zip(db_docs["documents"], db_docs["metadatas"]):
    bm_doc.append(Document(page_content=content, metadata=meta))

bm_doc[:3]

[Document(metadata={'creationdate': '2025-07-10T16:11:16+09:00', 'trapped': '/False', 'total_pages': 87, 'page': 0, 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'moddate': '2025-09-04T16:51:11+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'page_label': '1', 'producer': 'Adobe PDF Library 15.0'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'page_label': '2', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page': 1, 'creationdate': '2025-07-10T16:11:16+09:00', 'trapped': '/False', 'moddate': '2025-09-04T16:51:11+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'total_pages': 87, 'producer': 'Adobe PDF Library 15.0'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\

In [20]:
bm25 = BM25Retriever.from_documents(bm_doc)    # 키워드 기반 검색기
bm25.k = 5

In [25]:
# 벡터 검색기 + bm25 = 하이브리드 검색기 완성
ret_hybrid = EnsembleRetriever(
    retrievers = [ret_similarity, bm25],
    weights = [0.5, 0.5]
)

In [28]:
question = "삼성전자의 2025년 전망은?"
result = ret_hybrid.invoke(question)
result

[Document(id='f7c5dfd6-3ca9-4c9b-920c-d9b4472b9ccb', metadata={'producer': 'Adobe PDF Library 15.0', 'creationdate': '2025-07-10T16:11:16+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'page_label': '86', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'trapped': '/False', 'total_pages': 87, 'page': 85, 'moddate': '2025-09-04T16:51:11+09:00'}, page_content='삼성전자 지속가능경영보고서 2025\n86\n삼성전자주식회사는 경제·사회·환경적 가치 창출 성과를 다양한 이해관계자와 투명하게 소통하기 위해 2025년 열여덟 번째 지속가능경영보고서를 발간합니다.\n작성 기준\n본 보고서는 지속가능경영 보고 기준인 GRI(Global Reporting Initiative) \nStandard 2021에 따라 작성했습니다. 또한 글로벌 지속가능경영 \n표준과 이니셔티브인 UN SDGs(Sustainable Development Goals) \n와 TCFD(Task Force on Climate -related Financial Disclosures), \nSASB(Sustainability Accounting Standards Board)의 지표를 \n반영했습니다.\n보고 범위\n국내와 해 외 모든 사업장과 공급망을 포함합니다. 재무성과는 K-IFRS \n연결기준으로 작성했으며, 사업장 환경 성과는 국내 외 33개 생산 거점 \n뿐만 아니라 판매/연구소 등을 포함 하여 수집한 데이터를 기준으로 \n작성했습니다. 삼성전자의 본 점소재지는 경기도 수원시 영통구 삼성로 \n129(매탄동)에 위치해 있습니다.\n보고 기간\n2024년 1월1일

In [ ]:
rich_docs(result, title = "삼성전자 전망")

### 2-1) 압축 검색기 ( Compression Retriever)

- 검색된 문서가 길 때 -> llm을 이용해서 내용을 압축해보기
- 문서 내용이 너무 파편화 되어 있는 경우 -> 압축 진행 -> 찌꺼기가 제거됨 (띄어쓰기나 태그, 이모티콘 등)
- 문제점 : 비용문제 -> 각 문서별로 전부 압축을 진행해서 사용하기 때문

In [30]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name = "gpt-4.1-mini",
    temperature=0
)

In [31]:
# 압축기 생성
compressor = LLMChainExtractor.from_llm(model)

# 압축 검색기 생성 = 유사도 검색 -> 문서 내용 압축
comp_retriever = ContextualCompressionRetriever(
    base_retriever = ret_similarity,
    base_compressor = compressor
)

In [32]:
question = "삼성전자의 목표와 기준 년도만 간단히 알려줘"
comp_result = comp_retriever.invoke(question)
comp_result

[Document(metadata={'creationdate': '2025-07-10T16:11:16+09:00', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'page': 3, 'total_pages': 87, 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page_label': '4', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False'}, page_content="삼성전자는 2022년 9월 발표한 '新환경경영전략'을 기반으로 탄소중립 달성, 자원순환 극대화, 그리고 기술 혁신을 통한 환경 난제 해결을 위해 노력하고 있습니다.  \nDX(Device eXperience)부문은 2030년 탄소중립 달성을 목표로  \nDS(Device Solutions)부문은 2050년 탄소중립 달성을 목표로"),
 Document(metadata={'page_label': '86', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'producer': 'Adobe PDF Library 15.0', 'page': 85, 'trapped': '/False', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2025-09-04T16:51:11+09:00', 'creationdate': '2025-07-10T16:11:16+09:00', 'total_pages': 87}, page_content='삼성전자 지속가능경영보고서 2025  \n작성 기준  \n본 보고서는 지속가능경영 보고 기준인 GRI(Global Reporting Initiative)  \nStandard 2021에 따라 작성했

In [33]:
rich_docs(comp_result, title="삼성전자의 목표와 기준 년도")

                                            삼성전자의 목표와 기준 년도                                            
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자는 2022년 9월 발표한 '新환경경영전략'을   │
│   │                                                  │      │ 기반으로 탄소중립 달성, 자원순환 극대화, 그리고   │
│   │                                                  │      │ 기술 혁신을 통한 환경 난제 해결을 위해 노력하고   │
│   │                                                  │      │ 있습니다.   DX(Device eXperience)부문은 2030년    │
│   │                                                  │      │ 탄소중립 달성을 목표로   …                        │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025   작성 기준    … │
│   │                                                  │      │ 보고서는 지속가능경영 보고 기준인 GRI(Global      │
│   │                                                  │      │ Reporting Initiative)   Standard 2021에 따라      │
│   │                                                  │      │ 작성했습니다.    보고 기간   2024년 1월1일부터    │
│   │                                                  │      │ 2024…                                             │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025                  │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025                  │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하           │
│   │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품과  │
│   │                                                  │      │ 서비스를…                                         │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

### 2-2) 임베딩 기반 경량 압축 (비용 X)
- 잘 사용하진 않음
- 두개를 다 사용하고 싶을 때 (ex. 유사도 + 필터 / mmr + 필터)

In [35]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

emb_filter = EmbeddingsFilter(
    embeddings = embedding,
    similarity_threshold = 0.2
)
comp_embed = ContextualCompressionRetriever(
    base_retriever = ret_similarity,
    base_compressor = emb_filter
)
comp_embed_result = comp_embed.invoke(question)

In [36]:
rich_docs(comp_embed_result, title="임베딩 기반 필터기법")

                                               임베딩 기반 필터기법                                                
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ Our Company CEO 메시지 회사소개 기업 지배구조     │
│   │                                                  │      │ 중대성 평가 이해관계자 소통 04 05 06 …            │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하           │
│   │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품과  │
│   │                                                  │      │ 서비스를…                                         │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

## 5. Reranker

```
uv add sentence-transformers
uv add langchain_huggingface
```

- 우선 검색기로 후보군 추출 (ex. 10-30개)

In [ ]:
# 1. 유사도 기반으로 10개 추출
ret_similarity = load_vectorStore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 10}
)
question = "삼성과 현대 중에서 어느 기업이 더 나아?"
ret_similarity_result = ret_similarity.invoke(question)
rich_docs(ret_similarity_result, title="두 기업 비교")

```
uv add hf_xet
```

In [38]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

In [ ]:
# huggingFace model 가져오기
hf_ce = HuggingFaceCrossEncoder(
    model_name = "cross-encoder/ms-marco-MiniLM-L6-v2",
    model_kwargs = {
        "device" : "cuda",
        "max_length" : 512
    }
)

In [41]:
# 리랭크
compressor = CrossEncoderReranker(
    model = hf_ce,
    top_n = 10
)

reranker_retriever = ContextualCompressionRetriever(
    base_retriever = ret_similarity,
    base_compressor = compressor
)

reranker_result = reranker_retriever.invoke(question)
rich_docs(reranker_result, title = "리랭크 결과")

                                                    리랭크 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동규범 삼성전자는 제품과 서비스를 제공하는     │
│    │                                                  │      │ 모든 협력회사에 노동인권, 환경, 안전보건, 윤리   │
│    │                                                  │      │ 분야를 포함하는 현지 법규와    RBA(Responsible   │
│    │                                                  │      │ Business Alliance) 1)  행…                       │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경영 관리 수준 향상을 위한 실사와 개     │
│    │                                                  │      │ 선조치 이행, 정보공개 등의 성과는 해 당 부서의   │
│    │                                                  │      │ 경영진과 실무자들의 KPI로 반영되며 목표 달성은   │
│    │                                                  │      │ 급여, 인센티브 등 보상과 연계됩니다. 추진 방향   │
│    │                                                  │      │ 삼성전자…                                        │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke AI   │
│    │                                                  │      │ 로봇청소기 보안 인증 획득 2024년에는 Bespoke AI  │
│    │                                                  │      │ 스팀 로봇청소기가 KISA 개인정보보호중심설계      │
│    │                                                  │      │ (PbD, Privacy by Design) 인증과 KISA IoT 보안    │
│    │                                                  │      │ 인증 중 …                                        │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                              

## 6. Reorder

- 리랭크와 같이 사용
- 리랭크에서는 맥락(내용의 흐름) 고려 X

In [42]:
from langchain_community.document_transformers import LongContextReorder

reorder = LongContextReorder()
reordered_result = reorder.transform_documents(reranker_result)

rich_docs(reordered_result, title="리오더 결과")

                                                    리오더 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 재활용 소재를 적용하는 성과를 이뤘습니다.        │
│    │                                                  │      │ DS(Device Solutions)부문은 2050년 탄소중립       │
│    │                                                  │      │ 달성을 목표로  공정가스 통합 처리시설(RCS,       │
│    │                                                  │      │ Regenerative Catalytic System) 에 투자를         │
│    │                                                  │      │ 지속하고 있습니다. DS부문의 …                    │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ Our Company CEO 메시지 회사소개 기업 지배구조    │
│    │                                                  │      │ 중대성 평가 이해관계자 소통 04 05 06 …           │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   61 │ 삼성전자 지속가능경영보고서 2025 61 Facts &      │
│    │                                                  │      │ Figures  경제성과 사회성과 환경성과   사업부문 … │
│    │                                                  │      │ 환경성과  62 63 68 72 Our Company AppendixFacts  │
│    │                                                  │      │ & Figures PrinciplePlanet People                 │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                                  │      │ 글로벌 제조&인프라  총괄장인 CSO(Chief Safety    │
│    │                                                  │      │ Officer)를 중심으로 2030년까지 상주  협력회사    │
│    │                                                  │      │ 안전보건 역량과 관리체계 …                     